<a href="https://colab.research.google.com/github/thongle28625/ttnt/blob/main/BAOCAO1_MINIMAX_COCARO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math

class CoCaro:
    def __init__(self, kich_thuoc, quan_nguoi_dung):
        # Khởi tạo: Thừa nhận các thông số cơ bản của ván đấu
        self.n = kich_thuoc # Kích thước bàn cờ (n x n)
        self.quan_nguoi = quan_nguoi_dung.upper() # Quân của người chơi (X hoặc O)
        self.quan_ai = "O" if self.quan_nguoi == "X" else "X" #Chọn quân
        # Tạo ma trận bàn cờ trống bằng List Comprehension
        self.ban_co = [[None for _ in range(self.n)] for _ in range(self.n)]

        # Luân chuyển điều kiện thắng: n=3 cần 3 ô, n=4 cần 4 ô, n>=5 cần 5 ô.
        if self.n <= 3: self.k = 3
        elif self.n == 4: self.k = 4
        else: self.k = 5

    def nguoi_choi_hien_tai(self, ban_co):
        #Xác định lượt chơi: Kiểm tra xem đang là lượt của X hay O.
        # Đếm tổng số ô không trống
        tong_quan = sum(hang.count("X") + hang.count("O") for hang in ban_co)
        # Quy ước: X luôn đi trước
        return "X" if tong_quan % 2 == 0 else "O"

    def danh_sach_o_trong(self, ban_co):
        #Tìm tất cả tọa độ (hàng, cột) còn trống.
        return [(h, c) for h in range(self.n) for c in range(self.n) if ban_co[h][c] is None]

    def tao_trang_thai_moi(self, ban_co, hanh_dong):
        #Trả về một bản sao bàn cờ sau khi đánh thử một nước.
        p = self.nguoi_choi_hien_tai(ban_co)
        # Sao chép bàn cờ cũ sang bàn cờ mới để không làm hỏng dữ liệu gốc
        ban_co_moi = [hang[:] for hang in ban_co]
        ban_co_moi[hanh_dong[0]][hanh_dong[1]] = p
        return ban_co_moi

    def kiem_tra_nguoi_thang(self, ban_co):
        #Người thắng: Quét bàn cờ để tìm chuỗi k quân liên tiếp.
        huong = [(0, 1), (1, 0), (1, 1), (1, -1)] # Ngang, Dọc, Chéo xuôi, Chéo ngược
        for h in range(self.n):
            for c in range(self.n):
                if ban_co[h][c] is None: continue # Ô trống thì bỏ qua
                quan_tai_o = ban_co[h][c]
                for dh, dc in huong:
                    dem = 1 # Bắt đầu đếm từ ô hiện tại
                    for i in range(1, self.k):
                        nh, nc = h + dh * i, c + dc * i
                        # Nếu ô tiếp theo vẫn nằm trong bàn cờ và cùng loại quân
                        if 0 <= nh < self.n and 0 <= nc < self.n and ban_co[nh][nc] == quan_tai_o:
                            dem += 1
                        else: break
                    if dem == self.k: return quan_tai_o # Đủ số lượng liên tiếp => Thắng
        return None

    def ket_thuc_tran_dau(self, ban_co):
        #Trạng thái cuối: Trả về True nếu trận đấu đã dừng (có người thắng hoặc hết chỗ).
        if self.kiem_tra_nguoi_thang(ban_co) is not None: return True
        # Kiểm tra xem còn ô trống nào không
        return all(all(o is not None for o in hang) for hang in ban_co)

    def diem_so(self, ban_co):
        #Tính điểm cho AI (X thắng = 1, O thắng = -1).
        ket_qua = self.kiem_tra_nguoi_thang(ban_co)
        if ket_qua == "X": return 1
        if ket_qua == "O": return -1
        return 0 # Nếu hòa

    def thuat_toan_minimax(self, ban_co):
        # MINIMAX: Tìm nước đi thông minh nhất cho AI.
        p_hien_tai = self.nguoi_choi_hien_tai(ban_co)
        # Giới hạn độ sâu để tránh quá tải khi n lớn (n=3 nhìn 6 bước, n>3 nhìn 4 bước)
        do_sau_toi_da = 6 if self.n == 3 else 4

        def cat_tia_alpha_beta(trang_thai, do_sau, alpha, beta, dang_tim_max):
            # Hàm đệ quy Alpha-Beta Pruning: Cắt tỉa các nhánh không cần thiết để chạy nhanh hơn.
            if self.ket_thuc_tran_dau(trang_thai) or do_sau == 0:
                return self.diem_so(trang_thai)

            if dang_tim_max: # Tìm nước đi có điểm cao nhất
                v = -math.inf
                for hanh_dong in self.danh_sach_o_trong(trang_thai):
                    v = max(v, cat_tia_alpha_beta(self.tao_trang_thai_moi(trang_thai, hanh_dong), do_sau - 1, alpha, beta, False))
                    alpha = max(alpha, v)
                    if beta <= alpha: break # Cắt tỉa
                return v
            else: # Tìm nước đi có điểm thấp nhất
                v = math.inf
                for hanh_dong in self.danh_sach_o_trong(trang_thai):
                    v = min(v, cat_tia_alpha_beta(self.tao_trang_thai_moi(trang_thai, hanh_dong), do_sau - 1, alpha, beta, True))
                    beta = min(beta, v)
                    if beta <= alpha: break  # Cắt tỉa
                return v
        # Khởi chạy MINIMAX: Chọn nước đi tốt nhất dựa trên điểm số trả về từ alphabeta
        nuoc_di_tot_nhat = None
        if p_hien_tai == "X": # TRƯỜNG HỢP 1: AI là quân X (Người chơi muốn tăng điểm - Maximizing)
            gia_tri_lon_nhat = -math.inf
            for hanh_dong in self.danh_sach_o_trong(ban_co):
                gia_tri = cat_tia_alpha_beta(self.tao_trang_thai_moi(ban_co, hanh_dong), do_sau_toi_da, -math.inf, math.inf, False)
                if gia_tri > gia_tri_lon_nhat:
                    gia_tri_lon_nhat, nuoc_di_tot_nhat = gia_tri, hanh_dong
        else: # TRƯỜNG HỢP 2: AI là quân O (Người chơi muốn hạ điểm - Minimizing)
            gia_tri_nho_nhat = math.inf
            for hanh_dong in self.danh_sach_o_trong(ban_co):
                gia_tri = cat_tia_alpha_beta(self.tao_trang_thai_moi(ban_co, hanh_dong), do_sau_toi_da, -math.inf, math.inf, True)
                if gia_tri < gia_tri_nho_nhat:
                    gia_tri_nho_nhat, nuoc_di_tot_nhat = gia_tri, hanh_dong
        return nuoc_di_tot_nhat # Cuối cùng, trả về tọa độ (hàng, cột) tốt nhất đã tìm được

    def hien_thi_ban_co(self):
        #In ma trận ra màn hình.
        print("\n   " + "   ".join(map(str, range(self.n))))
        for i, hang in enumerate(self.ban_co):
            print(str(i) + " " + " | ".join([o if o else " " for o in hang]))

    def bat_dau_choi(self):
        #Chạy vòng lặp trò chơi.
        print(f"Chào mừng! Bàn cờ {self.n}x{self.n}, cần {self.k} quân liên tiếp để thắng.")
        while not self.ket_thuc_tran_dau(self.ban_co):
            self.hien_thi_ban_co()
            p = self.nguoi_choi_hien_tai(self.ban_co)

            if p == self.quan_nguoi:
                print(f"Lượt của bạn ({p})")
                try:
                    h, c = map(int, input("Nhập hàng,cột (vd: 1,2): ").split(','))
                    if self.ban_co[h][c] is None: self.ban_co[h][c] = p
                    else: print(">> Ô này đánh rồi!")
                except: print(">> Nhập sai định dạng!")
            else:
                print("AI đang tính toán...")
                nuoc_di = self.thuat_toan_minimax(self.ban_co)
                if nuoc_di: self.ban_co[nuoc_di[0]][nuoc_di[1]] = p

        self.hien_thi_ban_co()
        thang = self.kiem_tra_nguoi_thang(self.ban_co)
        if thang is None: print("HÒA CỜ!")
        else: print(f"{'BẠN' if thang == self.quan_nguoi else 'AI'} ĐÃ THẮNG!")

# CHƯƠNG TRÌNH CHÍNH
if __name__ == "__main__":
    n = int(input("Nhập kích thước bàn cờ: "))
    chon = input("Chọn X hoặc O: ")
    game = CoCaro(n, chon)
    game.bat_dau_choi()

Nhập kích thước bàn cờ: 3
Chọn X hoặc O: x
Chào mừng! Bàn cờ 3x3, cần 3 quân liên tiếp để thắng.

   0   1   2
0   |   |  
1   |   |  
2   |   |  
Lượt của bạn (X)
Nhập hàng,cột (vd: 1,2): 1,1

   0   1   2
0   |   |  
1   | X |  
2   |   |  
AI đang tính toán...

   0   1   2
0 O |   |  
1   | X |  
2   |   |  
Lượt của bạn (X)
Nhập hàng,cột (vd: 1,2): 0,2

   0   1   2
0 O |   | X
1   | X |  
2   |   |  
AI đang tính toán...

   0   1   2
0 O |   | X
1   | X |  
2 O |   |  
Lượt của bạn (X)
Nhập hàng,cột (vd: 1,2): 1,0

   0   1   2
0 O |   | X
1 X | X |  
2 O |   |  
AI đang tính toán...

   0   1   2
0 O |   | X
1 X | X | O
2 O |   |  
Lượt của bạn (X)
Nhập hàng,cột (vd: 1,2): 1,0
>> Ô này đánh rồi!

   0   1   2
0 O |   | X
1 X | X | O
2 O |   |  
Lượt của bạn (X)
Nhập hàng,cột (vd: 1,2): 0,1

   0   1   2
0 O | X | X
1 X | X | O
2 O |   |  
AI đang tính toán...

   0   1   2
0 O | X | X
1 X | X | O
2 O | O |  
Lượt của bạn (X)
Nhập hàng,cột (vd: 1,2): 2,2

   0   1   2
0 O | X | 